In [2]:
import pandas as pd
import numpy as np
import os
from typing import Callable # this is to define test_func functions dtype as function itself
import re # for regural expression
import warnings
warnings.filterwarnings('ignore')
import pickle # for pickling my model

# Can import the below dataset from 
/kaggle/input/mobile-recommendation-system-dataset/mobile_recommendation_system_dataset.csv

In [3]:
base_path = os.getcwd()
sub_path = 'data'
dataset_path = os.path.join(base_path, sub_path, 'mobile_recommendation_system_dataset.csv')

In [4]:
df = pd.read_csv(dataset_path,index_col=False)

In [5]:
df.head()

,name,ratings,price,imgURL,corpus
0,"REDMI Note 12 Pro 5G (Onyx Black, 128 GB)",4.2,23999,https://rukminim2.flixcart.com/image/312/312/x...,Storage128 GBRAM6 SystemAndroid 12Processor T...
1,"OPPO F11 Pro (Aurora Green, 128 GB)",4.5,"₹20,999",https://rukminim2.flixcart.com/image/312/312/k...,Storage128 GBRAM6 GBExpandable Storage256GB S...
2,"REDMI Note 11 (Starburst White, 64 GB)",4.2,13149,https://rukminim2.flixcart.com/image/312/312/x...,Storage64 GBRAM4 SystemAndroid 11Processor Sp...
3,"OnePlus Nord CE 5G (Blue Void, 256 GB)",4.1,21999,https://rukminim2.flixcart.com/image/312/312/x...,Storage256 GBRAM12 SystemAndroid Q 11Processo...
4,"APPLE iPhone 13 mini (Blue, 128 GB)",4.6,3537,https://rukminim2.flixcart.com/image/312/312/k...,Storage128 SystemiOS 15Processor TypeA15 Bion...


In [6]:
def random_values(col_name:str,total=6):
    for i in range(0,total):
        index = np.random.randint(df.shape[0])
        print(index,':',df[col_name].iloc[index])        

In [7]:
random_values('corpus')

109 : Storage128  SystemiOS 12Processor TypeA12 Bionic Chip 12MP 63MP 7MP 3G notMentioned Display Size15.49 cm (6.1 inch)Resolution1792 x 828 pixelsGPUApple-designed GPU - 4-core, Graphics Performance - Upto 50% Faster than A11 Bionic, Metal 2 Optimised, 3D, Graphics-intensive Gameplay, Faster and More FluidDisplay TypeAll-screen LCD Multi-touch Display with IPS TechnologyHD Game SupportYesOther Display Features1400:1 Contrast Ratio (Typical), True Tone Display (Six-channel Light Sensor), Wide Colour Display (P3), 625 nits Maximum Brightness (Typical), Fingerprint-resistant Oleophobic Coating, Support for Display of Multiple Languages and Characters Simultaneously, Liquid Retina HD Display, Tap to Wake, Wide Colour Gamut,
778 : Storage256 GBRAM12  SystemAndroid 12Processor TypeQualcomm Snapdragon 8+ Gen 1Processor Speed3.2 200MP 50MP 12MP 200MP 50MP 12MP 60MP 60MP 5G Capacity4610 Display Size16.94 cm (6.67 inch)Resolution2400 x 1080 PixelsResolution TypeFull HD+Display TypeFull HD+ pOL

## Lowercasing the corpus values

In [8]:
df['corpus'] = df['corpus'].str.lower()

In [9]:
random_values('corpus')

124 : storage64 gbram4 gbexpandable storage128gb  systemandroid marshmallow 6.0.1processor typequalcomm snapdragon 625 64-bit octa core 2ghzprocessor speed2 13mp 5mp 3g capacity4100 display size13.97 cm (5.5 inch)resolution1920 x 1080 pixelsresolution typefull hdgpuadreno 506display typeipsother display featurescontrast ratio: 1000:1, 2.5d glass, 72% ntsc color gamut, night display, reading mode, color temperature adjustment
672 : storage64  systemios 14processor typea14 bionic chip 12mp 12mp 12mp 12mp 12mp 5g notmentioned display size15.49 cm (6.1 inch)resolution2532 x 1170 pixelsresolution typesuper retina xdr displaydisplay typesuper retina xdr displayother display featuressuper retina xdr display, 6.1 inch (diagonal) all screen oled display, hdr display, true tone, wide colour (p3), haptic touch, 2000000:1 contrast ratio (typical), 625 nits max brightness (typical); 1200 nits max brightness (hdr), fingerprint-resistant oleophobic coating, support for display of multiple languages a

In [10]:
df.isnull().sum()

name        0
ratings     0
price       0
imgURL      0
corpus     12
dtype: int64

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2546 entries, 0 to 2545
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   name     2546 non-null   object 
 1   ratings  2546 non-null   float64
 2   price    2546 non-null   object 
 3   imgURL   2546 non-null   object 
 4   corpus   2534 non-null   object 
dtypes: float64(1), object(4)
memory usage: 99.6+ KB


In [12]:
df.dropna(inplace=True)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2534 entries, 0 to 2545
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   name     2534 non-null   object 
 1   ratings  2534 non-null   float64
 2   price    2534 non-null   object 
 3   imgURL   2534 non-null   object 
 4   corpus   2534 non-null   object 
dtypes: float64(1), object(4)
memory usage: 118.8+ KB


In [14]:
max_len = 0
for i in range(0, len(df['corpus'])):
    try:
        if len(df['corpus'][i]) > max_len:
            max_len = len(df['corpus'][i])
    except:
        pass

print(f'words max len in corpus is: {max_len}')


words max len in corpus is: 1196


In [15]:
df.sample(10)

,name,ratings,price,imgURL,corpus
1164,"OnePlus 10R 5G (Prime Blue, 128 GB)",4.0,31999,https://rukminim2.flixcart.com/image/312/312/x...,storage128 gbram8 systemandroid oxygen 12 48m...
1877,"SAMSUNG Galaxy A14 5G (Black, 64 GB)",4.1,15999,https://rukminim2.flixcart.com/image/312/312/x...,storage64 gbram4 gbtotal memory64gb systemand...
1151,"APPLE iPhone 12 Pro Max (Pacific Blue, 256 GB)",5.0,26273,https://rukminim2.flixcart.com/image/312/312/k...,storage256 systemios 14processor typea14 bion...
2424,"APPLE iPhone 13 (Pink, 512 GB)",4.5,26463,https://rukminim2.flixcart.com/image/312/312/k...,storage512 systemios 15processor typea15 bion...
347,"SAMSUNG Galaxy M14 5G (Smoky Teal, 128 GB)",4.2,14989,https://rukminim2.flixcart.com/image/312/312/x...,storage128 gbram4 systemandroid android 13.0p...
332,"OPPO Neo 7 4G (Black, 16 GB)",4.4,"₹8,000",https://rukminim2.flixcart.com/image/312/312/m...,storage16 gbram1 gbexpandable storage32gb sys...
389,"realme GT Master Edition (Luna White, 128 GB)",4.3,22999,https://rukminim2.flixcart.com/image/312/312/l...,storage128 gbram8 systemandroid 11processor t...
2453,"OPPO A54 (Starry Blue, 64 GB)",4.4,"₹13,990",https://rukminim2.flixcart.com/image/312/312/k...,storage64 gbram4 gbexpandable storage256gb sy...
1663,"Redmi Note 9 Pro (Glacier White, 128 GB)",4.3,17999,https://rukminim2.flixcart.com/image/312/312/k...,storage128 gbram6 gbexpandable storage512gb s...
1340,"APPLE iPhone 7 (Rose Gold, 128 GB)",4.4,30636,https://rukminim2.flixcart.com/image/312/312/k...,storage128 systemios 12processor typea10 fusi...


In [16]:
random_values('imgURL')

1310 : https://rukminim2.flixcart.com/image/312/312/l4ln8nk0/mobile/6/s/i/-original-imagfgf9nf8psuyv.jpeg?q=70
469 : https://rukminim2.flixcart.com/image/312/312/k2jbyq80pkrrdj/mobile-refurbished/z/a/f/iphone-11-pro-max-256-u-mwhm2hn-a-apple-0-original-imafkg2ftc5cze5n.jpeg?q=70
1550 : https://rukminim2.flixcart.com/image/312/312/kj0bp8w0-0/mobile/p/n/c/9-power-mobhqab5-redmi-original-imafyz7aqsrggzgh.jpeg?q=70
294 : https://rukminim2.flixcart.com/image/312/312/xif0q/mobile/6/x/a/11r-5g-5011102525-oneplus-original-imagn3afeqfr6acy.jpeg?q=70
525 : https://rukminim2.flixcart.com/image/312/312/xif0q/mobile/y/m/q/-original-imagcqr7spupzwtz.jpeg?q=70
1912 : https://rukminim2.flixcart.com/image/312/312/kyg5zm80/mobile/7/q/r/-original-imagazh6zbafgv8u.jpeg?q=70


# Model

## vectorization(to convert the words to vectors)   (BOW)

In [16]:
from sklearn.feature_extraction.text import CountVectorizer 

In [17]:
cont_vect = CountVectorizer(max_features=1000)

In [18]:
arrays = cont_vect.fit_transform(df['corpus'])

In [19]:
vectors = arrays.toarray()

In [20]:
cont_vect.get_feature_names_out()

array(['00', '000', '02', '03', '04', '05', '07', '08', '09', '0mp',
       '0primary', '0processor', '10', '100', '1000', '100000', '1000000',
       '100mp', '103', '104', '106', '1080', '1080processor',
       '1080resolution', '108mp', '10mp', '10processor', '11', '1100',
       '1125', '1170', '1179', '11primary', '11processor', '12', '120',
       '1200', '1200processor', '120hz', '1242', '1280processor', '1284',
       '1290', '12mp', '12primary', '12processor', '13', '1300', '1330',
       '1334', '135', '13mp', '13processor', '14', '1400', '144', '1440',
       '144hz', '1480', '14processor', '15', '1500', '1544',
       '15processor', '16', '160', '1600', '16000', '1600resolution',
       '1612', '16mp', '16processor', '18', '180', '1800', '180hz', '19',
       '1920', '1920hz', '1processor', '20', '200', '2000000', '200mp',
       '20mp', '21', '22', '23', '2340', '24', '240', '2400',
       '2400resolution', '2408', '240hz', '2460', '24mp', '25mm', '25mp',
       '26', '27'

## TFIDF to create the vector

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
TFIDF = TfidfVectorizer(max_features = 100, ngram_range = (2,2)) #here binary BOW is applied

In [40]:
vectors = TFIDF.fit_transform(df['corpus']).toarray()

In [41]:
vectors[0]

array([0.        , 0.16679359, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.26334773, 0.        ,
       0.        , 0.        , 0.        , 0.2959421 , 0.        ,
       0.23352892, 0.        , 0.        , 0.2269421 , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.14455869, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.12099622, 0.        , 0.18371565, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.53714322, 0.        , 0.        , 0.        , 0.20151362,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.16441008, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.20212233, 0.        , 0.        , 0.        , 0.     

# WORD2VEC
### This creates the vector of each and every word not for the whose sentences like BOW and TFIDF

In [54]:
!pip install gensim


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [36]:
df.index

Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
       ...
       2535, 2536, 2537, 2538, 2539, 2540, 2541, 2542, 2544, 2545],
      dtype='int64', length=2534)

In [38]:
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
corpus = []
for i in df.index:
    description = df['corpus'][i]
    description = description.split()
    
    description = [lemmatizer.lemmatize(word) for word in description]
    description = ' '.join(description)
    corpus.append(description)
    

In [40]:
corpus

['storage128 gbram6 systemandroid 12processor typemediatek dimensity 1080processor speed2.6 50mp 8mp 2mp 50mp 16mp 5g capacity5000 display size16.94 cm (6.67 inch)resolution2400 x 1080 pixelsresolution typefull hd+ amoled displaygpuarm mali-g68 mc4display typefull hd+ amoled displayother display featuresrefresh rate: 120 hz, contrast: 5,000,000:1, 1920hz pwm dimming, brightness level: 16000:1, peak brightness: 900 nit',
 'storage128 gbram6 gbexpandable storage256gb systemandroid pie 9.0processor typemediatek helio p70 octa core 2.1 ghzprocessor speed2.1 48mp 5mp 48mp 5mp 48mp 5mp 48mp 5mp 48mp 5mp 16mp 3g capacity4000 display size16.51 cm (6.5 inch)resolution2340 x 1080 pixelsresolution typefull hd+gpuarm mali-g72 mp3 at 900 mhzdisplay colors16mother display featuresscreen texture: tft-ltps, in-cell touch panel technology, narrow frame: 1.70mm (size from aa to outside edge of glass), screen ratio: 90.90%, screen contrast: 1500:1 (typical), color saturation: ntsc 81.5% (typical), maximu

In [41]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [42]:
corpus[0]

'storage128 gbram6 systemandroid 12processor typemediatek dimensity 1080processor speed2.6 50mp 8mp 2mp 50mp 16mp 5g capacity5000 display size16.94 cm (6.67 inch)resolution2400 x 1080 pixelsresolution typefull hd+ amoled displaygpuarm mali-g68 mc4display typefull hd+ amoled displayother display featuresrefresh rate: 120 hz, contrast: 5,000,000:1, 1920hz pwm dimming, brightness level: 16000:1, peak brightness: 900 nit'

In [43]:
simple_preprocess(corpus[0])

['storage',
 'gbram',
 'systemandroid',
 'processor',
 'typemediatek',
 'dimensity',
 'processor',
 'speed',
 'mp',
 'mp',
 'mp',
 'mp',
 'mp',
 'capacity',
 'display',
 'size',
 'cm',
 'inch',
 'resolution',
 'typefull',
 'hd',
 'amoled',
 'displaygpuarm',
 'mali',
 'mc',
 'display',
 'typefull',
 'hd',
 'amoled',
 'displayother',
 'display',
 'featuresrefresh',
 'rate',
 'hz',
 'contrast',
 'hz',
 'pwm',
 'dimming',
 'brightness',
 'level',
 'peak',
 'brightness',
 'nit']

In [50]:
words=[]
for sent in corpus:
    sent_token=sent_tokenize(sent)
    for sent in sent_token:
        words.append(simple_preprocess(sent))

In [52]:
words[0]

['storage',
 'gbram',
 'systemandroid',
 'processor',
 'typemediatek',
 'dimensity',
 'processor',
 'speed',
 'mp',
 'mp',
 'mp',
 'mp',
 'mp',
 'capacity',
 'display',
 'size',
 'cm',
 'inch',
 'resolution',
 'typefull',
 'hd',
 'amoled',
 'displaygpuarm',
 'mali',
 'mc',
 'display',
 'typefull',
 'hd',
 'amoled',
 'displayother',
 'display',
 'featuresrefresh',
 'rate',
 'hz',
 'contrast',
 'hz',
 'pwm',
 'dimming',
 'brightness',
 'level',
 'peak',
 'brightness',
 'nit']

In [54]:
from gensim.models import Word2Vec


# Train a Word2Vec model
model = Word2Vec(words, vector_size=100, window=5, min_count=1, workers=4)

In [55]:
model.wv['dimensity']

array([-0.06992002, -0.5114751 , -0.20268169,  0.31355053, -0.59616315,
       -0.23265357, -0.02694047,  0.04478803,  0.0141142 , -0.5577411 ,
        0.66170716,  0.744044  , -0.5410456 , -0.31992474, -0.01647909,
        0.08877818,  0.46188945,  0.2618103 , -0.5004267 ,  0.06314929,
        0.19754606, -0.03895353,  0.6119809 , -0.14539903,  0.2854034 ,
       -0.07822682, -0.09406967,  0.42080057,  0.45323688, -0.01760698,
        0.01037227, -0.18177488,  0.23951326, -0.18063053,  0.21092455,
       -0.3616809 , -0.1830549 ,  0.7061192 ,  0.13034624,  0.22385795,
        0.12689522,  0.28008422, -0.16701013, -0.23444013,  0.11990002,
       -0.01000222, -0.58474624,  0.69904244, -0.08602739,  0.48722485,
       -0.69188803,  0.4471588 , -0.15515275, -0.20915578, -0.24690512,
       -0.39259374,  0.02487046, -0.1593265 ,  0.37429097,  0.2091256 ,
       -0.4599599 ,  0.11769563,  0.45728645, -0.283498  , -0.30336478,
       -0.01166587,  0.11647913,  0.580564  , -0.39001104, -0.23

In [56]:
model.wv.index_to_key

['mp',
 'display',
 'storage',
 'processor',
 'inch',
 'resolution',
 'size',
 'cm',
 'capacity',
 'speed',
 'systemandroid',
 'gbram',
 'ratio',
 'hd',
 'typefull',
 'screen',
 'brightness',
 'hz',
 'gbexpandable',
 'rate',
 'touch',
 'typehd',
 'contrast',
 'gb',
 'nit',
 'to',
 'colors',
 'color',
 'features',
 'displayother',
 'typical',
 'body',
 'typemediatek',
 'retina',
 'typequalcomm',
 'snapdragon',
 'lcd',
 'displaydisplay',
 'amoled',
 'glass',
 'refresh',
 'xdr',
 'typesuper',
 'wide',
 'max',
 'aspect',
 'nits',
 'hdr',
 'and',
 'support',
 'tb',
 'coating',
 'oleophobic',
 'helio',
 'systemios',
 'ntsc',
 'pixel',
 'mother',
 'mali',
 'resistant',
 'fingerprint',
 'typea',
 'chip',
 'bionic',
 'for',
 'game',
 'gpuarm',
 'mode',
 'sampling',
 'ip',
 'true',
 'of',
 'colour',
 'tone',
 'notmentioned',
 'saturation',
 'gpuadreno',
 'simultaneously',
 'character',
 'language',
 'multiple',
 'pie',
 'in',
 'gorilla',
 'maximum',
 'haptic',
 'typ',
 'gamut',
 'octa',
 'cell',

In [57]:
def avg_word2vec(doc):
    return np.mean([model.wv[word] for word in doc if word in model.wv.index_to_key],axis=0)

In [58]:
from tqdm import tqdm

In [59]:
words[0]

['storage',
 'gbram',
 'systemandroid',
 'processor',
 'typemediatek',
 'dimensity',
 'processor',
 'speed',
 'mp',
 'mp',
 'mp',
 'mp',
 'mp',
 'capacity',
 'display',
 'size',
 'cm',
 'inch',
 'resolution',
 'typefull',
 'hd',
 'amoled',
 'displaygpuarm',
 'mali',
 'mc',
 'display',
 'typefull',
 'hd',
 'amoled',
 'displayother',
 'display',
 'featuresrefresh',
 'rate',
 'hz',
 'contrast',
 'hz',
 'pwm',
 'dimming',
 'brightness',
 'level',
 'peak',
 'brightness',
 'nit']

In [60]:
#apply for the entire sentences
X=[]
for i in tqdm(range(len(words))):
    X.append(avg_word2vec(words[i]))

100%|██████████| 2573/2573 [00:00<00:00, 3326.77it/s]


In [61]:
X[0]

array([-6.90593600e-01, -1.81337837e-02, -3.77089046e-02,  2.91062891e-01,
       -6.49958670e-01,  2.73572598e-02,  4.99515176e-01, -2.70931274e-01,
       -1.76573247e-01, -6.46728277e-01,  5.23778677e-01,  4.81843919e-01,
       -7.56445527e-01, -4.05444205e-01, -6.53857052e-01, -6.32624179e-02,
        7.94332981e-01,  2.88050890e-01, -3.59116346e-01, -4.07650292e-01,
        1.73512831e-01,  8.09149668e-02,  5.75917661e-01,  3.77114683e-01,
        5.27327001e-01,  2.57900298e-01, -4.67929125e-01,  1.02524780e-01,
        7.90314153e-02,  2.15036757e-02, -2.78220683e-01,  1.87637195e-01,
        1.01576686e-01, -6.66717768e-01,  6.92309663e-02, -1.31886974e-01,
       -3.13587755e-01,  4.31791395e-01,  3.54559928e-01,  8.13438520e-02,
        1.50817633e-01,  1.82571948e-01, -1.55611321e-01, -2.56259739e-01,
        3.63327056e-01, -2.17069432e-01, -1.05384362e+00,  5.22683978e-01,
        4.19567861e-02,  3.57304096e-01, -5.51852047e-01,  6.64753914e-01,
       -7.15819523e-02, -

## Calculating cosine similarity for recommendation

In [62]:
from sklearn.metrics.pairwise import cosine_similarity

In [63]:
# similarity = cosine_similarity(vectors)
similarity = cosine_similarity(X)
similarity.shape # this is because I have calculated the similarity among all the rows to all the rows(confusing?? but try to understand)

(2573, 2573)

In [64]:
similarity[2]

array([0.841445  , 0.74627674, 0.99999964, ..., 0.99999964, 0.9581574 ,
       0.97079706], dtype=float32)

In [45]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:11]

[(68, np.float64(1.0000000000000002)),
 (303, np.float64(1.0000000000000002)),
 (457, np.float64(0.947383121118287)),
 (1027, np.float64(0.947383121118287)),
 (1735, np.float64(0.947383121118287)),
 (616, np.float64(0.9270943960367082)),
 (1461, np.float64(0.9270943960367082)),
 (1123, np.float64(0.8572705626301631)),
 (1540, np.float64(0.8572705626301631)),
 (569, np.float64(0.8290718896419462))]

In [68]:
def recommend(mobile):
    mobile_index = df[df['name']==mobile].index[0]
    similarity_array = similarity[mobile_index]
    similar_10_mobiles = sorted(list(enumerate(similarity_array)),reverse=True,key=lambda x:x[1])[1:11]
    
    for i in similar_10_mobiles:
        print(df['name'].iloc[i[0]])
        print(df['corpus'].iloc[i[0]])



In [69]:
df['name'][0:10]

0      REDMI Note 12 Pro 5G (Onyx Black, 128 GB)
1            OPPO F11 Pro (Aurora Green, 128 GB)
2         REDMI Note 11 (Starburst White, 64 GB)
3         OnePlus Nord CE 5G (Blue Void, 256 GB)
4            APPLE iPhone 13 mini (Blue, 128 GB)
5             APPLE iPhone 6s Plus (Gold, 64 GB)
6         REDMI 10 Power (Sporty Orange, 128 GB)
7          Infinix Hot 11 (Emerald Green, 64 GB)
8    Infinix Note 10 Pro (Nordic Secret, 256 GB)
9             SAMSUNG Galaxy A04 (Green, 128 GB)
Name: name, dtype: object

In [71]:
recommend('APPLE iPhone 13 mini (Blue, 128 GB)')

vivo X90 Pro (Legendary Black, 256 GB)
storage256 gbram12  systemandroid 13processor typedimensity 9200processor speed3.05 50mp 50mp 12mp 50mp 50mp 12mp 32mp 32mp 2g capacity4870 display size17.22 cm (6.78 inch)resolution2800 x 1260 pixelsresolution typefull hd+gpuimmortalis g715display typefull hd+ amoled displayother display featuresscreen to body ratio: 93.53%, aspect ratio: 20:9
APPLE iPhone 13 mini (Blue, 128 GB)
storage128  systemios 15processor typea15 bionic chip 12mp 12mp 12mp 63mp 12mp 12mp 5g notmentioned display size13.72 cm (5.4 inch)resolution2340 x 1080 pixelsresolution typesuper retina xdr displaydisplay typesuper retina xdr displayother display featuressuper retina xdr display, 5.4‑inch (diagonal) all‑screen oled display, hdr display, true tone, wide colour (p3), haptic touch, 20,00,000:1 contrast ratio (typical), 800 nits max brightness (typical), 1,200 nits max brightness (hdr), fingerprint-resistant oleophobic coating, support for display of multiple languages and c

## making Pickle file of df and similarities to use in web

In [23]:
os.makedirs(r'src\model',exist_ok=True)

In [24]:
pickle.dump(obj=df,file=open(file=r'src\model\dataframe.pkl',mode='wb'))

In [25]:
pickle.dump(obj=similarity,file=open(file=r'src\model\similarity.pkl',mode='wb'))